In [ ]:
# Install required packages (run once)
%pip install -q gradio transformers torch

In [ ]:
# Import libraries and load the summarization model
import gradio as gr
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Using T5-small model with explicit model and tokenizer loading
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
def summarize_text(text: str) -> str:
    """Summarize the input `text` using the T5 model.
    Handles edge cases like empty or very short input.
    """
    if not text or text.strip() == "":
        return "(No input provided)"

    # Prepare input for T5 model
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary with beam search and prevent n‑gram repetition
    summary_ids = model.generate(
        inputs,
        max_length=150,
        min_length=40,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3,
        do_sample=False,
    )

    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary.strip()

In [ ]:
# Build the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## Text Summarizer")

    input_text = gr.Textbox(
        lines=8,
        label="Input Text",
        placeholder="Paste text to summarize..."
    )

    summarize_btn = gr.Button("Summarize")

    summary_output = gr.Textbox(
        lines=5,
        label="Summary"
    )

    summarize_btn.click(
        fn=summarize_text,
        inputs=input_text,
        outputs=summary_output
    )

demo.launch()